In [1]:
import spotipy
sp = spotipy.Spotify()
from spotipy.oauth2 import SpotifyClientCredentials 
import spotipy.util as util
from pprint import pprint
import matplotlib.pyplot as plt 
import numpy as np
import string

cid ="36748baa9fbd45e7885d6b90137a46a7" 
secret = "ad562808e82d4184a360347d848e050e"
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(cid, secret))

In [7]:
marra = 'spotify:artist:5AZuEF0feCXMkUCwQiQlW7'
gue =   'spotify:artist:7F2utINZ6tSokSiZTQBE27'
jake =  'spotify:artist:6JFRI91YaCXREGQYzHSnUH'
fedez = 'spotify:artist:3pgCLfNbw5ozIfoNsvDU7i'
baby_gang = 'spotify:artist:3LvwPiJQJ0da0GurKMToV0'
geolier = 'spotify:artist:27LlKWxS3KXW7RRAxN5S8s'
tony = 'spotify:artist:6CKch2otN4SPznHf9ms5JF'
parcels = 'spotigy:artist:3oKRxpszQKUjjaHz388fVA'

In [175]:
marra_plot_clean= featuring_clean(marra)
marra_plot= featuring(marra)
gue_plot = featuring_clean(gue)
fedez_plot = featuring_clean(fedez)
jake_plot = featuring_clean(jake)
geo_plot = featuring_clean(geolier)

In [5]:
def clean_album(artist_id):
    album_names = []
    all_albums = spotify.artist_albums(artist_id, album_type='album')
    
    #uri delle vari album
    for album in all_albums['items']:
        album_names.append([album['name'], album['uri']])

    rip = []
    titles = []
    for album in album_names:
        titles.append([album[0].translate(str.maketrans('', '', string.punctuation)), album[1]])
    titles.sort(key=lambda s: len(s[0]))
    for i in range(len(titles)):
        for b in range(len(titles)):
            if(i < b):
                if(set(titles[i][0].split(' ')).issubset(set(titles[b][0].split(' '))) == True):
                    rip.append(b)
    rip = list(set(rip))
    for ele in sorted(rip, reverse = True):
        del titles[ele]
    return titles 

In [4]:
def featuring_clean(artist_id):
    album_uri = []
    all_albums= clean_album(artist_id)
    
    #uri delle vari album
    for album in all_albums:
        album_uri.append(album[1])
    
    #nome dell'artista
    artist_name = spotify.artist(artist_id)['name']
    
    dict_feat = {}
    
    for i in album_uri:
        results = spotify.album_tracks(i)
        for tracks in results['items']:
            for b in tracks['artists']:
                if(b['name'] != artist_name):
                    if b['name'] in dict_feat.keys():
                        dict_feat[b['name']] += 1
                    else:
                        dict_feat[b['name']] = 1
                        
    dict_feat = dict(sorted(dict_feat.items(), key=lambda item: item[1]))
    
    return dict_feat, artist_name

In [6]:
def featuring(artist_id):
    album_uri = []
    all_albums = spotify.artist_albums(artist_id, album_type='album')
    
    #uri delle vari album
    for album in all_albums['items']:
        album_uri.append(album['uri'])
    
    #nome dell'artista
    artist_name = spotify.artist(artist_id)['name']
    
    dict_feat = {}
    
    for i in album_uri:
        results = spotify.album_tracks(i)
        for tracks in results['items']:
            for b in tracks['artists']:
                if(b['name'] != artist_name):
                    if b['name'] in dict_feat.keys():
                        dict_feat[b['name']] += 1
                    else:
                        dict_feat[b['name']] = 1
                        
    dict_feat = dict(sorted(dict_feat.items(), key=lambda item: item[1]))
    
    return dict_feat, artist_name

In [3]:
def appears_on_album(artist_id):
    
    appear_feat = {}

    all_album = spotify.artist_albums(artist_id, album_type='appears_on', limit = 50)
    for album in all_album['items']:
        if(album['album_type'] == 'album'):
            if album['artists'][0]['name'] in appear_feat.keys():
                appear_feat[album['artists'][0]['name']] += 1
            else:
                appear_feat[album['artists'][0]['name']] = 1
            
    return appear_feat


In [2]:
def plot_histo(voc, dimension):
    plt.rcdefaults()
    fig, ax = plt.subplots(figsize = dimension)

# Example data
    y_pos = np.arange(len(list(voc[0].keys())))
    ax.barh(y_pos, voc[0].values(), align='center', color = 'green')
    ax.set_yticks(y_pos, labels=list(voc[0].keys()))
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_xlabel('Number of feats')
    ax.set_title(voc[1] + ' ' + 'feats')